# Main code to run SSH through Python

In [ ]:
# Using SSH via Python

import paramiko
from getpass import getpass
import os
import sys

# Create the file prompt
theFile = input("Please enter the desired file for the output ")

# Little test to make sure there is actual input
if theFile == "":
    sys.exit("No response detected. Stopping code.")
else:
    pass

# Check that theFile exists in local directory
if os.path.exists(theFile) == True:
    changeFile = input('The specified file already exists.  |  Would you like to replace this file removing the previous one?(yes/no) ')

    # Following if,elif,else statement determines what to do with theFile based on users input
    if changeFile.lower() == "no":
        sys.exit("Okay the file will remain intact and the code will stop.")

    elif changeFile.lower() == "yes":
        os.remove(theFile)
        pass

    else:
        sys.exit("No response detected. Stopping code.")
        
# If it does not exist continue like usual
else:
    pass

# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host information
host = "192.168.6.71"
port = 2222
username = "christopher.mchugh"
password = thePass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed.")



### Send a file to remote device

In [ ]:
# Start SFTP session
sftp = ssh.open_sftp()

# User variables to make code more applicable
theLocation = input("What is the full path to the file you want to transfer? ")
theName = input("What would you like the name of this file to be on the remote device? ")

# Upload the file to the remote device
sftp.put(theLocation,'/home/christopher.mchugh/'+theName)

### Run a command using sudo

In [ ]:
# loads sftp
sftp = ssh.open_sftp()

# gathers user input
command = input("What command would you like to run with elevated permission?")


# Get the output from the command
stdin, stdout, stderr = ssh.exec_command('sudo -S '+command)

# used to pass through password for sudo and 'hit' enter
stdin.write(thePass+"\n")

# Get results from stdout
lines = stdout.readlines()

# Convert the list to a string
output = ''.join(lines)

# Save the output to a file
with open(theFile, 'a') as f:
    f.write(output)

#ssh.close()

### Outputs the contents of the home directory

In [ ]:
# Loads sftp
sftp = ssh.open_sftp()

# stores the command
command = 'sudo -S dir'

# Get the output from the command
stdin, stdout, stderr = ssh.exec_command(command)

# input password to run command as sudo, including a newline
# character that provides "Enter" as if you were typing it on a keyboard
stdin.write(thePass+"\n")

# Get results from stdout
lines = stdout.readlines()
print(lines)

# Convert the list to a string
output = ''.join(lines)

# Save the output to a file
with open(theFile, 'a') as f:
    f.write(output)

### Downloads File


In [ ]:
# Loads sftp
sftp = ssh.open_sftp()

# creates user variables 
downFile = input('What is the full path of the file you wish to download? ')
nameFile = input ('What do you want to name the file? ')

# Gets the file
sftp.get(downFile,'Suspicious/'+nameFile)

### Remove File from Home directory

In [ ]:
# Start sftp session
sftp = ssh.open_sftp()

# User input for file to remove
rmFile = input('What file would you like to remove? ')

# command
removecommand = 'sudo -S rm -f '+rmFile

# Get the output from the command
stdin, stdout, stderr = ssh.exec_command(removecommand)

# input password to run command as sudo, including a newline
# character that provides "Enter" as if you were typing it on a keyboard
stdin.write(thePass+"\n")

#ssh.close()


### Search through specified files for all exploitation commands

In [ ]:
import re
# All exploitation commands
cmds = {'blind_files' : ['/etc/resolv.conf','/etc/issue','etc/passwd','/etc/shadow','/home/'+username+'.bash_history'],
        'system' : ['uname -a','ps aux','id','arch','who -a','perl -v','df -k','mount','last -a','lastlog','dmesg','lspci','lscpu','lshw','/proc/cpuinfo','/proc/meminfo'],
        'networking' : ['hostname -f','ip addr show','ip ro show','iptables -L -n -v','iptables-save',' /proc/net/*'],
        'user_accounts' : ['/etc/passwd','/etc/shadow','/etc/group','getemt passwd','getent group'],
        'obt_user_info' : ['ls -alh /home/*/','ls -alh /home/*/.ssh/','/home/*/.viminfo','sudo -l','crontab -l'],
        'credentials' : ['/home/*/.ssh/id*','/tmp/krb5cc_*','/tmp/krb5.keytab','/home/*/.gnupg/secring.gpgs'],
        'configs' : ["ls -aRl /etc/ * awk '$1 ~ /w.$/' * grep -v lwrx 2> /dev/nullte",'/etc/group','/etc/hosts','/etc/crontab','/etc/sysctl.conf','/etc/resolv.conf','/etc/ldap/ldap.conf','cat /etc/fstab'],
        'determine_distro' : ['uname -a','lsb_release -d','/etc/os-release','/etc/issue','/etc/*release','/etc/debian_version','arch'],
        'installed_packages' : ['dpkg -l','dpkg -l | grep -i "linux-image"','dpkg --get-selections'],
        'package_sources' : ['/etc/apt/sources.list'],
        'find_important_files' : ['ls -alR | grep ^d','find /var -type -d','ls -dl `find /var -type d`','find /var -type d` | grep -v root','find /var ! -user root -type d -ls','find / -perm -4000','ls -alhtr /mnt','ls -alhtr /media','ls -alhtr /tmp','ls -alhtr /home',"find /home -type f -iname '.*history'","find /sbin /user/sbin /opt /lib `echo $PATH | 'sed s/:/ /g'` -perm /6000 -ls"],
}

# List of the three files to search through
files = ['deepDive/auth','deepDive/ps','deepDive/passwd']

# loop through the files listed above
for file in files:
        # Open a file
    with open(file) as f:

        # read in the file and save it to a variable
        contents = f.readlines()

    # Lists to store the results
    results = []
    # Loop through the list returned. Each element is a line from the smallSyslog file
    for line in contents:

        # The or if and for below are used to work with values inside of the dictionary.
        for key, values in cmds.items():
                #print (key)
                if(isinstance(values, list)):
                        for value in values:
                                #print (value)

                                # Search for the value in the line
                                x = re.findall(r''+value+'', line)

                                # For each value found in the above 
                                for found in x:

                                        # Append the returned keywords the the results list
                                        results.append(line)
                else:
                        print(value)
# Return results
print(results)